# 5 Slepian Symmetry

## Bessel Function

Bessel Function of the First Kind, J_m(k), for integer m 

J_m(k) = sum_{n=0}^{∞} [ (-1)^n / (n! (m+n)!) ] * (k/2)^(m+2n) 

Also satisfies the symmetry condition J_{-m}(k) = (-1)^m J_m(k).\\

In [9]:
using SpecialFunctions  # for besselj

"""
    identity_sum_J(k, M)

Compute the partial sum S_M = J_0^2(k) + 2∑_{m=1 to M} J_m^2(k)
and compare it to 1. As M -> ∞, we expect S_M -> 1.
"""
function identity_sum_J(k::Real, M::Integer)
    term0 = besselj(0, k)^2
    # sum over m = 1..M
    term_sum = sum( besselj(m, k)^2 for m in 1:M )
    return term0 + 2*term_sum
end

"""
    bessel_expansion_2D(k, r, rprime, theta, thetaprime, M)

Compute the partial sum of 
  J_0(k||x - x'||)  ≈  J_0(kr) J_0(kr') + 2∑_{m=1..M} J_m(kr) J_m(kr') cos[m(θ - θ')].
Returns a tuple (lhs, rhs_M):

- lhs = J_0(k * distance(x, x'))
- rhs_M = partial sum up to m = M.
  
As M grows, rhs_M should converge to lhs.
"""
function bessel_expansion_2D(k::Real,
                             r::Real, rprime::Real,
                             theta::Real, thetaprime::Real,
                             M::Integer)
    # LHS: exact Bessel function on the distance
    # distance in Cartesian: x   = (r cosθ,   r sinθ)
    #                       x'  = (r'cosθ', r' sinθ')
    # norm(x - x') = sqrt(r^2 + r'^2 - 2 r r' cos(θ - θ'))
    dist = sqrt(r^2 + rprime^2 - 2r*rprime*cos(theta - thetaprime))
    lhs = besselj(0, k * dist)
    
    # RHS: partial sum
    rhs0 = besselj(0, k*r) * besselj(0, k*rprime)
    rhs_sum = 0.0
    for m in 1:M
        rhs_sum += besselj(m, k*r)*besselj(m, k*rprime)*cos(m*(theta - thetaprime))
    end
    rhs_M = rhs0 + 2*rhs_sum
    
    return lhs, rhs_M
end

Check symmetry condition J_{-m}(k) = (-1)^m J_m(k):
m = -2, difference = 0.0
m = -1, difference = 0.0
m = 0, difference = 0.0
m = 1, difference = 0.0
m = 2, difference = 0.0

Check the sum identity S_M = J_0^2(k) + 2∑_{m=1..M} J_m^2(k), expecting it → 1
k = 0.0, M = 20, partial sum = 1.0
k = 1.0, M = 20, partial sum = 0.9999999999999999999999999999999999999999999999999998296138925678890401766074983
k = 5.0, M = 20, partial sum = 0.999999999999999999999964307560465423684089034832761010233377166154966369493576
k = 10.0, M = 20, partial sum = 0.9999999999820720147344384145125405414225483093980344286401843866624406690762224

Check 2D expansion J_0(k||x-x'||) with M=10
 - LHS = -0.2533057497160424370632976814177157851111228433892295300590824470557805147113848
 - RHS (partial sum up to m=10) = -0.2533057431388011234044304023961841205773432251380264121586651813690459929024044
 - Difference = -6.577241313658867279021531664533779618251203117900417265686734521808980395896989e-09


### Sanity check for Bessel Function

In [10]:
"""
    check_symmetry(m, k)

Check numerically that J_{-m}(k) = (-1)^m * J_m(k).
Returns the difference left - right for a sanity check.
"""
function check_symmetry(m::Integer, k::Real)
    left  = besselj(-m, k)
    right = (-1)^m * besselj(m, k)
    return left - right
end

println("Check symmetry condition J_{-m}(k) = (-1)^m J_m(k):")
for m in -2:2
    diff = check_symmetry(m, 5.0)
    println("m = $m, difference = $(diff)")
end
println()

# 2) Check the identity 1 = J_0^2(k) + 2∑_{m=1 to ∞} J_m^2(k)
# We'll do a partial sum up to M = 20 for some k values:
kvals = [0.0, 1.0, 5.0, 10.0]
M = 20
println("Check the sum identity S_M = J_0^2(k) + 2∑_{m=1..M} J_m^2(k), expecting it → 1")
for k in kvals
    s_M = identity_sum_J(k, M)
    println("k = $k, M = $M, partial sum = $s_M")
end
println()

# 3) Check the 2D expansion
# Suppose r=1, r'=2, theta=0.1 rad, theta'=1.0 rad, k=3. We'll do partial sums up to M=10:
r, rprime = 1.0, 2.0
theta, thetaprime = 0.1, 1.0
k = 3.0
M = 10
lhs, rhs_M = bessel_expansion_2D(k, r, rprime, theta, thetaprime, M)
println("Check 2D expansion J_0(k||x-x'||) with M=$M")
println(" - LHS = $lhs")
println(" - RHS (partial sum up to m=$M) = $rhs_M")
println(" - Difference = ", lhs - rhs_M)

Check symmetry condition J_{-m}(k) = (-1)^m J_m(k):
m = -2, difference = 0.0
m = -1, difference = 0.0
m = 0, difference = 0.0
m = 1, difference = 0.0
m = 2, difference = 0.0

Check the sum identity S_M = J_0^2(k) + 2∑_{m=1..M} J_m^2(k), expecting it → 1
k = 0.0, M = 20, partial sum = 1.0
k = 1.0, M = 20, partial sum = 0.9999999999999999999999999999999999999999999999999998296138925678890401766074983
k = 5.0, M = 20, partial sum = 0.999999999999999999999964307560465423684089034832761010233377166154966369493576
k = 10.0, M = 20, partial sum = 0.9999999999820720147344384145125405414225483093980344286401843866624406690762224

Check 2D expansion J_0(k||x-x'||) with M=10
 - LHS = -0.2533057497160424370632976814177157851111228433892295300590824470557805147113848
 - RHS (partial sum up to m=10) = -0.2533057431388011234044304023961841205773432251380264121586651813690459929024044
 - Difference = -6.577241313658867279021531664533779618251203117900417265686734521808980395896989e-09
